In [13]:
"""
Dedalus script simulating 2D horizontally-periodic Rayleigh-Benard convection.
This script demonstrates solving a 2D Cartesian initial value problem. It can
be ran serially or in parallel, and uses the built-in analysis framework to save
data snapshots to HDF5 files. The `plot_snapshots.py` script can be used to
produce plots from the saved data. It should take about 5 cpu-minutes to run.

For incompressible hydro with two boundaries, we need two tau terms for each the
velocity and buoyancy. Here we choose to use a first-order formulation, putting
one tau term each on auxiliary first-order gradient variables and the others in
the PDE, and lifting them all to the first derivative basis. This formulation puts
a tau term in the divergence constraint, as required for this geometry.

To run and plot using e.g. 4 processes:
    $ mpiexec -n 4 python3 rayleigh_benard.py
    $ mpiexec -n 4 python3 plot_snapshots.py snapshots/*.h5
"""


'\nDedalus script simulating 2D horizontally-periodic Rayleigh-Benard convection.\nThis script demonstrates solving a 2D Cartesian initial value problem. It can\nbe ran serially or in parallel, and uses the built-in analysis framework to save\ndata snapshots to HDF5 files. The `plot_snapshots.py` script can be used to\nproduce plots from the saved data. It should take about 5 cpu-minutes to run.\n\nFor incompressible hydro with two boundaries, we need two tau terms for each the\nvelocity and buoyancy. Here we choose to use a first-order formulation, putting\none tau term each on auxiliary first-order gradient variables and the others in\nthe PDE, and lifting them all to the first derivative basis. This formulation puts\na tau term in the divergence constraint, as required for this geometry.\n\nTo run and plot using e.g. 4 processes:\n    $ mpiexec -n 4 python3 rayleigh_benard.py\n    $ mpiexec -n 4 python3 plot_snapshots.py snapshots/*.h5\n'

In [14]:
import numpy as np
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)
import copy
import h5py
import numpy as np
import matplotlib
import re

import matplotlib.pyplot as plt
from dedalus.extras import plot_tools

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize

import os
from os import listdir

In [15]:
# Parameters
Lx, Lz = 20,1
Nx, Nz = 640, 32
Ra_M = 4.5e5
D_0 = 0
D_H = 1/3
M_0 = 0
M_H = -1
N_s2=4/3
f=0.1

Prandtl = 0.7
dealias = 3/2
stop_sim_time = 100
timestepper = d3.RK222
max_timestep = 0.125
dtype = np.float64

In [16]:
# Bases
coords = d3.CartesianCoordinates('x','z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
zbasis = d3.ChebyshevT(coords['z'], size=Nz, bounds=(0, Lz), dealias=dealias)

In [17]:
# Fields
p = dist.Field(name='p', bases=(xbasis,zbasis))
D = dist.Field(name='D', bases=(xbasis,zbasis))
M = dist.Field(name='M', bases=(xbasis,zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis,zbasis))
uy = dist.Field(name='uy', bases=(xbasis,zbasis))
Z = dist.Field(name='Z', bases=zbasis)
tau_p = dist.Field(name='tau_p')
tau_B1 = dist.Field(name='tau_B1', bases=xbasis)
tau_B2 = dist.Field(name='tau_B2', bases=xbasis)
tau_D1 = dist.Field(name='tau_D1', bases=xbasis)
tau_D2 = dist.Field(name='tau_D2', bases=xbasis)
tau_M1 = dist.Field(name='tau_M1', bases=xbasis)
tau_M2 = dist.Field(name='tau_M2', bases=xbasis)
tau_u1 = dist.VectorField(coords,name='tau_u1', bases=xbasis)
tau_u2 = dist.VectorField(coords,name='tau_u2', bases=xbasis)
tau_u3 = dist.VectorField(coords,name='tau_u3', bases=xbasis)
tau_u4 = dist.Field(name='tau_u4', bases=xbasis)

# Substitutions    
#Kuo_Bretherton Equilibrium
kappa = (Ra_M * Prandtl/((M_0-M_H)*Lz**3))**(-1/2)
nu = (Ra_M / (Prandtl*(M_0-M_H)*Lz**3))**(-1/2)
print('kappa',kappa)
print('nu',nu)


x,z = dist.local_grids(xbasis,zbasis)
Z['g']=z
Z.change_scales(3/2)

ex,ez = coords.unit_vector_fields(dist)
lift_basis = zbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)

B_op = (np.absolute(D - M - N_s2*Z)+ M + D - N_s2*Z)/2

Max = lambda A,B: (abs(A-N_s2*Z-B)+A-N_s2*Z+B)/2
eva = lambda A: A.evaluate()

dz= lambda A: d3.Differentiate(A, coords['z'])
dx= lambda A: d3.Differentiate(A, coords['x'])

ux=u@ex
uz=u@ez
dxux=dx(ux)
dzux=dz(ux)
dxuz=dx(uz)
dzuz=dz(uz)

grad_u = d3.grad(u) + ez* lift(tau_u1) # First-order reduction
grad_ux = grad_u@ex # First-order reduction
grad_uz = grad_u@ez # First-order reduction
grad_M = d3.grad(M) + ez*lift(tau_M1) # First-order reduction
grad_D = d3.grad(D) + ez*lift(tau_D1) # First-order reduction

kappa 0.001781741612749496
nu 0.0012472191289246471


In [18]:
# Problem
# First-order form: "div(f)" becomes "trace(grad_f)"
# First-order form: "lap(f)" becomes "div(grad_f)"
problem = d3.IVP([p, M, D, u, uy, tau_p, tau_M1, tau_M2, tau_D1, tau_D2, tau_u1, tau_u2], namespace=locals())
problem.add_equation("trace(grad_u) + tau_p= 0")
problem.add_equation("dt(M) - kappa*div(grad_M) + lift(tau_M2) = - u@grad(M)")
problem.add_equation("dt(D) - kappa*div(grad_D) + lift(tau_D2) = - u@grad(D)")
problem.add_equation("dt(ux) + dx(p) - nu*div(grad_ux) + lift(tau_u2)@ex = - u@grad(ux)+f*uy")
problem.add_equation("dt(uz) + dz(p) - nu*div(grad_uz) + lift(tau_u2)@ez = - u@grad(uz) + B_op")
problem.add_equation("dt(uy) - nu*div(grad(uy)) = -u@grad(uy) -f*ux")
problem.add_equation("u(z=0) = 0")
problem.add_equation("u(z=Lz) = 0")
problem.add_equation("M(z=0) = M_0")
problem.add_equation("D(z=0) = D_0")
problem.add_equation("M(z=Lz) = M_H")
problem.add_equation("D(z=Lz) = D_H")
problem.add_equation("integ(p) = 0") # Pressure gauge

{'LHS': Integrate(Integrate(<Field 5340981200>)),
 'RHS': 0,
 'condition': 'True',
 'tensorsig': (),
 'dtype': numpy.float64,
 'M': 0,
 'L': Integrate(Integrate(<Field 5340981200>)),
 'F': <Field 5351575632>,
 'domain': <dedalus.core.domain.Domain at 0x13efdda90>,
 'matrix_dependence': array([ True,  True]),
 'matrix_coupling': array([False,  True])}

In [19]:
# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time


2023-09-22 10:43:31,201 subsystems 0/1 INFO :: Building subproblem matrices 1/320 (~0%) Elapsed: 0s, Remaining: 27s, Rate: 1.2e+01/s
2023-09-22 10:43:32,747 subsystems 0/1 INFO :: Building subproblem matrices 32/320 (~10%) Elapsed: 2s, Remaining: 15s, Rate: 2.0e+01/s
2023-09-22 10:43:34,364 subsystems 0/1 INFO :: Building subproblem matrices 64/320 (~20%) Elapsed: 3s, Remaining: 13s, Rate: 2.0e+01/s
2023-09-22 10:43:36,025 subsystems 0/1 INFO :: Building subproblem matrices 96/320 (~30%) Elapsed: 5s, Remaining: 11s, Rate: 2.0e+01/s
2023-09-22 10:43:37,652 subsystems 0/1 INFO :: Building subproblem matrices 128/320 (~40%) Elapsed: 7s, Remaining: 10s, Rate: 2.0e+01/s
2023-09-22 10:43:39,451 subsystems 0/1 INFO :: Building subproblem matrices 160/320 (~50%) Elapsed: 8s, Remaining: 8s, Rate: 1.9e+01/s
2023-09-22 10:43:41,054 subsystems 0/1 INFO :: Building subproblem matrices 192/320 (~60%) Elapsed: 10s, Remaining: 7s, Rate: 1.9e+01/s
2023-09-22 10:43:41,158 subsystems 0/1 INFO :: Building

In [20]:
# Initial condition
D.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise
D['g'] *= z * (Lz - z) # Damp noise at walls
D['g'] += (D_H-D_0)*z # Add linear background
M.fill_random('g', seed=28, distribution='normal', scale=1e-3) # Random noise
M['g'] *= z * (Lz - z) # Damp noise at walls
M['g'] += (M_H-M_0)*z # Add linear background

In [21]:
# Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', sim_dt=0.25, max_writes=50)
snapshots.add_tasks(solver.state,layout='g')

In [22]:
# CFL
CFL = d3.CFL(solver, initial_dt=0.1, cadence=10, safety=0.5, threshold=0.05,
             max_change=1.1, min_change=0.5, max_dt=max_timestep)
CFL.add_velocity(u)

In [23]:
# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=10)
flow.add_property(np.sqrt(u@u)/nu, name='Re')


In [24]:
# Main loop
startup_iter = 10
try:
    logger.info('Starting main loop')
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if (solver.iteration-1) % 10 == 0:
            max_Re = flow.max('Re')
            logger.info('Iteration=%i, Time=%e, dt=%e, max(Re)=%f' %(solver.iteration, solver.sim_time, timestep, max_Re))
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()

2023-09-22 10:43:47,522 __main__ 0/1 INFO :: Starting main loop
2023-09-22 10:43:48,303 __main__ 0/1 INFO :: Iteration=1, Time=1.000000e-01, dt=1.000000e-01, max(Re)=0.000000
2023-09-22 10:43:49,121 __main__ 0/1 INFO :: Iteration=11, Time=1.100000e+00, dt=1.000000e-01, max(Re)=0.211175
2023-09-22 10:43:50,252 __main__ 0/1 INFO :: Iteration=21, Time=2.200000e+00, dt=1.100000e-01, max(Re)=693.284150
2023-09-22 10:43:51,350 __main__ 0/1 INFO :: Iteration=31, Time=2.750000e+00, dt=5.500000e-02, max(Re)=nan
2023-09-22 10:43:52,158 __main__ 0/1 INFO :: Iteration=41, Time=3.300000e+00, dt=5.500000e-02, max(Re)=nan
2023-09-22 10:43:52,944 __main__ 0/1 INFO :: Iteration=51, Time=3.850000e+00, dt=5.500000e-02, max(Re)=nan
2023-09-22 10:43:53,721 __main__ 0/1 INFO :: Iteration=61, Time=4.400000e+00, dt=5.500000e-02, max(Re)=nan
2023-09-22 10:43:54,499 __main__ 0/1 INFO :: Iteration=71, Time=4.950000e+00, dt=5.500000e-02, max(Re)=nan
2023-09-22 10:43:55,299 __main__ 0/1 INFO :: Iteration=81, Time=

KeyboardInterrupt: 